In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

from depmapomics import tracker as track
from depmapomics import expressions

import dalmatian as dm
from gsheets import Sheets
import pandas as pd

MY_ID = '~/.client_secret.json'
MYSTORAGE_ID = "~/.storage.json"

Sheets.from_files(MY_ID, MYSTORAGE_ID)
#autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
# loading annotations
CCLE_annotation = track.getTracker() # the function uses pygsheets to load this: REFSHEET_URL=https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY
# Sheets.from_files(MY_ID, MYSTORAGE_ID).get(REFSHEET_URL).sheets[0].to_frame(index_col=0)
# you can also get it from pd.read_csv('gs://ccle_default_params/celligner_ex/CCLE_annotation.csv.gz', index_col=0)

In [ ]:
CCLE_annotation = CCLE_annotation.rename(columns=rename)[['sample_id', 'participant_id', 'sex', 'age', 'primary_site', 'disease_type', "disease_subtype", 'origin', 'mediatype', 'sequencing_type']]
CCLE_annotation['method']="bulk"
CCLE_annotation['cell_type']="historical_CL"

In [ ]:
val[val.issue!=""].issue

In [ ]:
val = h.makeCellosaurusExport()

In [ ]:
val

In [ ]:
anc = h.getAncestry(val)

In [ ]:
a = ACH.drop_duplicates('arxspan_id', keep='first').set_index("arxspan_id")[['RRID', "Primary/Metastasis", "Adult/Pediatric", "CCLF Age"]]

In [ ]:
b = CCLE_annotation.drop_duplicates('arxspan_id', keep='first').set_index("arxspan_id")[['cellosaurus_id', 'age']]

In [ ]:
na = pd.concat([a, b], axis=1)

In [ ]:
na.loc[~na.RRID.isna()&((na.cellosaurus_id=="U") | na.cellosaurus_id.isna()), "cellosaurus_id"] = na[~na.RRID.isna()&((na.cellosaurus_id=="U") | na.cellosaurus_id.isna())].RRID.tolist()

In [ ]:
na.loc[na.RRID.isna()&~na.cellosaurus_id.isna(), "RRID"] = na[na.RRID.isna()&~na.cellosaurus_id.isna()].cellosaurus_id.tolist()

In [ ]:
na.loc[~((na["CCLF Age"]=='Unknown') | na["CCLF Age"].isna()) & (na.age.isna() | (na.age=="U")), "age"] = na[~((na["CCLF Age"]=='Unknown') | na["CCLF Age"].isna()) & (na.age.isna() | (na.age=="U"))]["CCLF Age"].tolist()

In [ ]:
na.loc[((na["CCLF Age"]=='Unknown') | na["CCLF Age"].isna()) & ~(na.age.isna() | (na.age=="U")), "CCLF Age"] = na[((na["CCLF Age"]=='Unknown') | na["CCLF Age"].isna()) & ~(na.age.isna() | (na.age=="U"))]["age"].tolist()

In [ ]:
# ACH-002050 adult 2years
# many age are actually months
na.loc["ACH-000644", "age"] = "45"
na.loc["ACH-003019", "age"] = "75"
na.loc["ACH-002050", "Adult/Pediatric"] = "Pediatric"

In [ ]:
na.loc[~((na["Adult/Pediatric"]=='-1') | na["Adult/Pediatric"].isna()) & (na.age.isna() | (na.age=="U")), "age"] = na[~((na["Adult/Pediatric"]=='-1') | na["Adult/Pediatric"].isna()) & (na.age.isna() | (na.age=="U"))]["Adult/Pediatric"].tolist()

In [ ]:
na

In [ ]:
for i, va in na.iterrows():
    if va.cellosaurus_id is not None and va.cellosaurus_id!='U':
        if va.cellosaurus_id in val.index:
            if val.loc[va.cellosaurus_id, 'isMeta'] != (va["Primary/Metastasis"]=="Metastasis"):
                if val.loc[va.cellosaurus_id, 'isMeta']:
                    na.loc[i,"Primary/Metastasis"]="Metastasis"

In [ ]:
toadd = val[val.depmap_id.isin(set(val[~val.depmap_id.isna() & (val.depmap_id!='')].depmap_id)- set(na[~na.cellosaurus_id.isna()&(na.cellosaurus_id!="U")].index))]

In [ ]:
na.loc[toadd.depmap_id.tolist(), 'cellosaurus_id'] = toadd.index.tolist()

In [ ]:
na

In [ ]:
subset = (~na.cellosaurus_id.isna()&(na.cellosaurus_id!="U") & ~na.cellosaurus_id.isin(['CVCL_4J73', 'CVCL_Y549', 'CVCL_UF98', 'CVCL_3280', 'CVCL_M266', 'CVCL_8722']))

In [ ]:
na.loc[na[subset].index.tolist(), ["characteristics", "ancestry", "issue", "instability", "transfected", "synonyms", "cellosaurus_comments","atcclink","dsmzlink","doublingt","hasebv", "cellosaurus_origin"]] = val.loc[na[subset].cellosaurus_id.tolist(), ["characteristics","ancestry","issue", "instability","transfected", "synonyms", "comments","atcclink","dsmzlink","doublingt","hasebv", "origin"]].values

In [ ]:
CCLE_annotation[["Primary/Metastasis", "characteristics", "ancestry", "issue", "instability", "transfected", "synonyms", "cellosaurus_comments","atcclink","dsmzlink","doublingt","hasebv", "cellosaurus_origin"]] = None

In [ ]:
tofix = ["age", "cellosaurus_id", "Primary/Metastasis", "characteristics", "ancestry", "issue", "instability", "transfected", "synonyms", "cellosaurus_comments","atcclink","dsmzlink","doublingt","hasebv", "cellosaurus_origin"]
for i, val in na.iterrows():
    CCLE_annotation.loc[CCLE_annotation.arxspan_id==i, tofix] = val[tofix].values

In [ ]:
from gsheets import Sheets
from depmapomics.config_prod import MY_ID, MYSTORAGE_ID
otherano = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/1LYY2gWVnNzjGtM68TBgvA7fpN51T-5OJD2ShvQRmfsM/").sheets[0].to_frame(index_col=0)#gid=1776180249
b = otherano.drop_duplicates('arxspan_id', keep='first').set_index("arxspan_id")[['primary_site']]

na = pd.concat([a, b], axis=1)

In [ ]:
set(na[~(na['Sample Collection Site'].isna() | (na['Sample Collection Site']=="-1"))]['Sample Collection Site'])

In [ ]:
ina = (na["primary_site"].isna() | (na['primary_site']=="U")) & ~(na['Sample Collection Site'].isna() | (na['Sample Collection Site']=="-1"))
#inb = ~(na["primary_site"].isna() | (na['primary_site']=="U")) & (na['Sample Collection Site'].isna() | (na['Sample Collection Site']==-1))
na.loc[ina, "primary_site"] = na.loc[ina, "Sample Collection Site"] 
#na.loc[inb, "Sample Collection Site"] = na.loc[inb, "primary_site"] 

In [ ]:
na.loc["ACH-001743", "Sample Collection Site"] = "bone_marrow"

In [ ]:
na.loc[na["Sample Collection Site"]!=na['primary_site'], "primary_site"]
na.loc[na["Sample Collection Site"]!=na['primary_site'], "primary_site"] = na.loc[na["Sample Collection Site"]!=na['primary_site'], "Sample Collection Site"].values

In [ ]:
na = na.replace({'primary_site':{'-1': 'U',
 'Colon': 'colon',
 'Embryonal': 'embryo',
 'Other': 'U',
 'Placenta': 'placenta',
 'Testes': 'testes',
 'Thymus': 'thymus',
 'Unknown': 'U',
 'brain': 'central_nervous_system',
 'engineered': 'U',
 'left heel':'heel',
 'matched_normal_tissue': 'U',
 'pericardial_effusion': 'pleural_effusion',
 'pleura': "pleural_effusion",
 'pleural_cavity': "pleural_effusion",
 'right heel': 'heel',
}})

In [ ]:
for i, v in na['primary_site'].iteritems():
    CCLE_annotation.loc[CCLE_annotation.arxspan_id==i, 'collection_site'] = v

In [ ]:
dfToSheet(CCLE_annotation, "ccle sample tracker", "../.credentials.json")